In [34]:
import psycopg2
from psycopg2.extensions import AsIs
import psycopg2
import pandas as pd
import numpy as np
import shapely
from shapely import wkb
from shapely.geometry import mapping, shape
from flask import Flask, jsonify, request, after_this_request

In [35]:

db_credentials = {"dbname": 'gta',
                  "port": 5432,
                  "user": 'gta_p8',
                  "password": 'r7sdkfdq',
                  "host": 'ikgpgis.ethz.ch'}

In [36]:
#fiktiver input von flask
trip_id = 10

In [37]:
def get_linestring(trip_id, db_credentials):
    trip_id = str(trip_id)
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()
    sql_string = "SELECT geometry FROM gta_p8.trip WHERE trip_id='"+trip_id+"';"
    cur.execute(sql_string)
    wkb_hex = cur.fetchone()
    conn.commit()
    conn.close()
    linestring = wkb.loads(bytes.fromhex(wkb_hex[0]))
    return linestring


In [38]:
def buffering_linestring(linestring, distance=15.0/1000):
    return linestring.buffer(distance)

In [39]:
def get_point_ids(buffer_polygon, db_credentials):
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()
    sql_string = "SELECT restaurant_id FROM gta_p8.restaurant WHERE ST_Contains(ST_GeomFromText('"+str(buffer_polygon)+"', 4326), geometry);"
    cur.execute(sql_string)
    list = cur.fetchone()
    return list


In [40]:
def get_id_list(trip_id, db_credentials):
    trip_id = str(trip_id)
    conn = psycopg2.connect(**db_credentials)
    cur = conn.cursor()
    sql_string = "SELECT restaurant_id FROM gta_p8.restaurant JOIN gta_p8.trip ON gta_p8.trip.trip_id = 10 WHERE ST_Contains(ST_Buffer(ST_Transform(gta_p8.trip.geometry, 3857),100, 'endcap=flat join=round'),ST_Transform(gta_p8.restaurant.geometry, 3857));"
    cur.execute(sql_string)
    list = cur.fetchall()
    conn.commit()
    conn.close()
    return list

In [44]:
def convert_to_json(list):
    array = []
    for i in list:
        array.append(i[0])
    return array

In [45]:
list = get_id_list(trip_id, db_credentials)

In [46]:

json = convert_to_json(list)

[(975,), (1264,)]
[975, 1264]


In [ ]:
sql_string = "SELECT * FROM gta_p8.restaurant JOIN gta_p8.trip ON gta_p8.trip.trip_id = 10 WHERE ST_Contains(ST_Buffer(ST_Transform(gta_p8.trip.geometry, 3857),100, 'endcap=flat join=round'),ST_Transform(gta_p8.restaurant.geometry, 3857));"